In [10]:
import os
import biom
from biom import Table
import pandas as pd
import numpy as np

In [54]:
metadata = pd.read_excel("../../data/metadata.xls")

In [55]:
metadata.head()

,Run,study,subject_id,treatment,group1,group2,group,timepoint,timepoint_numeric,timepoint_id,fber_type,fber_amount,time_days,age,sex,bmi,PCR_primers,Region,Platform,Country
0,SRR6444565,PRJNA428736,U222,control,Control,Control,0.0,2016-09-01 00:00:00,1.0,U222_1,accessible,NaN,0,17-29,NaN,NaN,FWD:GTGCCAGCMGCCGCGGTAA;REV:GGACTACHVGGGTWTCTAAT,V4,Illumina MiSeq,North America
1,SRR6443884,PRJNA428736,U222,control,Control,Control,0.0,2016-09-01 00:00:00,1.0,U222_1,accessible,NaN,0,17-29,NaN,NaN,FWD:GTGCCAGCMGCCGCGGTAA;REV:GGACTACHVGGGTWTCTAAT,V4,Illumina MiSeq,North America
2,SRR6444382,PRJNA428736,U222,control,Control,Control,0.0,2016-09-01 00:00:00,1.0,U222_1,accessible,NaN,0,17-29,NaN,NaN,FWD:GTGCCAGCMGCCGCGGTAA;REV:GGACTACHVGGGTWTCTAAT,V4,Illumina MiSeq,North America
3,SRR6443860,PRJNA428736,U222,control,Control,Control,0.0,2016-10-01 00:00:00,2.0,U222_2,accessible,NaN,14,17-29,NaN,NaN,FWD:GTGCCAGCMGCCGCGGTAA;REV:GGACTACHVGGGTWTCTAAT,V4,Illumina MiSeq,North America
4,SRR6444083,PRJNA428736,U222,control,Control,Control,0.0,2016-10-01 00:00:00,2.0,U222_2,accessible,NaN,14,17-29,NaN,NaN,FWD:GTGCCAGCMGCCGCGGTAA;REV:GGACTACHVGGGTWTCTAAT,V4,Illumina MiSeq,North America


In [56]:
study = metadata.study.unique()

In [57]:
study

array(['PRJNA428736', 'PRJNA780023', 'PRJNA560950', 'PRJEB2165',
       'PRJNA293971', 'PRJNA306884', 'PRJNA385004', 'PRJNA891951',
       'PRJEB41443'], dtype=object)

In [58]:
for i in study:
    df = metadata.loc[metadata.study.values == i]
    dir_path = "/home/dongbiao/word_embedding_microbiome/programe_test/dietary_fber/data/projects"
    os.makedirs(f"{dir_path}/{i}", exist_ok=True)
    df["Run"].to_csv(f"{dir_path}/{i}/sample.txt", index=False, header=False)

In [59]:
n = 1 
for name in study:
    if n == 1:
        table = biom.load_table(f"/home/dongbiao/word_embedding_microbiome/programe_test/dietary_fber/data/projects/{name}/results/extracted-feature-table/feature-table.biom")
        reads_depth = table.sum(axis="sample")
        sid = table.ids(axis="sample")
        table.filter(sid[reads_depth > 1000], axis='sample')
        table.remove_empty()
        fid = table.ids(axis="observation")
        prevalence = table.nonzero_counts(axis="observation")
        table.filter(fid[prevalence > 1], axis="observation")
        table.remove_empty()
    else:
        temp = biom.load_table(f"/home/dongbiao/word_embedding_microbiome/programe_test/dietary_fber/data/projects/{name}/results/extracted-feature-table/feature-table.biom")
        reads_depth = temp.sum(axis="sample")
        sid = temp.ids(axis="sample")
        temp.filter(sid[reads_depth > 1000], axis='sample')
        temp.remove_empty()
        fid = temp.ids(axis="observation")
        prevalence = temp.nonzero_counts(axis="observation")
        temp.filter(fid[prevalence > 1], axis="observation")
        
        table = table.merge(temp)
    n = n + 1
table.remove_empty()

6721 x 2382 <class 'biom.table.Table'> with 314215 nonzero entries (1% dense)

In [60]:
reads_depth = table.sum(axis="sample")
sid = table.ids(axis="sample")
table.filter(sid[reads_depth > 1000], axis='sample')
table.remove_empty()

6721 x 2378 <class 'biom.table.Table'> with 314204 nonzero entries (1% dense)

In [61]:
with biom.util.biom_open(f"/home/dongbiao/word_embedding_microbiome/programe_test/dietary_fber/data/projects/table_{table.shape[0]}_{table.shape[1]}.biom", 'w') as f:
    table.to_hdf5(f, 'example')

In [62]:
metadata = metadata.loc[[i in table.ids(axis="sample") for i in metadata.Run.values]]

In [63]:
for i in study:
    sid = metadata.loc[metadata.study.values == i].Run.values
    temp = table.filter(sid, axis="sample", inplace=False)
    temp.remove_empty()
    print(temp.shape)
    print(i)

(1964, 1176)
PRJNA428736
(1334, 248)
PRJNA780023
(1037, 197)
PRJNA560950
(37, 34)
PRJEB2165
(1449, 60)
PRJNA293971
(954, 153)
PRJNA306884
(198, 74)
PRJNA385004
(4033, 318)
PRJNA891951
(675, 118)
PRJEB41443
